In [1]:
from FxDerivative import Vanilla
import datetime as dt
from RateHandle import rate 

In [26]:
strike = 1.18
date = dt.date(2026, 5, 5)
option_type = 'call'
N = 100000

option = Vanilla()

c:\Users\S542282\Desktop\fede_env\Lib\site-packages\scipy\optimize\_slsqp_py.py:435: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
c:\Users\S542282\Desktop\fede_env\Lib\site-packages\scipy\optimize\_slsqp_py.py:435: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
c:\Users\S542282\Desktop\fede_env\Lib\site-packages\scipy\optimize\_slsqp_py.py:435: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
c:\Users\S542282\Desktop\fede_env\Lib\site-packages\scipy\optimize\_slsqp_py.py:435: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
c:\Users\S542282\Desktop\fede_env\Lib\site-packages\scipy\optimize\_slsqp_py.py:435: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)


In [27]:
option.contract(option_type, strike, date, N)

print('Ref:', option.ref, 'option price:', round(option.Premium_EUR(),2), 'EUR')

Ref: 1.15065 option price: 2326.96 EUR


In [8]:
target_date = ql.Date(18, 6, 2027)

In [14]:
option.mkt.sofr.rate_curve.zeroRate(target_date, ql.Actual365Fixed(), ql.Continuous, ql.Annual).rate()

0.037834027777777786

In [19]:
import QuantLib as ql

# ---------- 1. Set evaluation context ----------
today = ql.Date(18, 6, 2025)          # or whatever your "as-of" date is
ql.Settings.instance().evaluationDate = today
calendar = ql.TARGET()                # EUR calendar; use UnitedStates() if you prefer

# ---------- 2. Raw market data ----------
# >>> Replace these with *your* actual dates & forward quotes <<<
expiry_dates = [
    ql.Date(18, 7, 2025),             # 1-month
    ql.Date(18, 9, 2025),             # 3-month
    ql.Date(18,12, 2025),             # 6-month
    ql.Date(18, 6, 2026),             # 1-year
    ql.Date(18, 6, 2027)              # 2-year
]
forward_quotes = [1.0720, 1.0745, 1.0790, 1.0905, 1.1150]

# ---------- 3. Choose day-count & interpolation ----------
day_counter   = ql.Actual365Fixed()   # most common for FX forwards
interpolator  = ql.Linear()           # piece-wise linear in time on the *rates*
# For log-linear (smooth on ln(rate)) use ql.LogLinear()

# ---------- 4. Build the forward term structure ----------
fx_fwd_curve = ql.ForwardCurve(
    expiry_dates,
    forward_quotes,
    day_counter,
    calendar,          # optional in recent QuantLib builds; safe to include

)



In [24]:
fx_fwd_curve.zeroRate(ql.Date(18, 8, 2025), day_counter, ql.Continuous, ql.Annual).rate()

1.0744999999999993

In [6]:
from RateHandle import rate 
import QuantLib as ql

In [3]:
a = rate()

In [4]:
a.data_take('USD')

In [14]:
a.rate_interpolator(ql.Date(20, 6, 2025))

RuntimeError: negative time (-0.0277778) given